Mathematics of Functional Programming
=====================================

Algebraic Structure
-------------------

- Morphisms
- Objects

### Morphisms

Think functions. These are interactions between objects.

### Objects

Think types.

More About Morphisms
--------------------

### Operations ###

The domains and the range are the same set.

In [1]:
type Operation<T> = (...operands: T[]) => T;

'use strict'

In [2]:
const isOperation = <T>(operation: Operation<T>) => true;

undefined

#### Example ####

In [3]:
const increment = (n: number) => n + 1;

undefined

In [4]:
isOperation(increment);

true

#### Counter-Example ####

In [5]:
const strLength = (s: string) => s.length;

undefined

In [ ]:
isOperation(strLength);

Error: Line 1, Character 13
isOperation(strLength);
____________^
TS2345: Argument of type '(s: string) => number' is not assignable to parameter of type 'Operation<string>'.
  Type 'number' is not assignable to type 'string'.

### Dyadic Operations ###

AKA Binary Operations.

$$a,b \in M \implies a \odot b \in M$$

In [7]:
type DyadicOperation<T> = (a: T, b: T) => T;

undefined

#### Example ####

Addition of natural numbers.

In [8]:
const add = (a: number, b: number) => a + b;

undefined

Algebraic Structures
====================

- Objects
- Morphisms
- Morphism Laws

Magma
-----

![Liquid Hot Magma](./liquid-hot-magma.gif)

Magma Axioms
------------

Totality.

In [9]:
type Magma<T> = DyadicOperation<T>;

undefined

Ro-Sham-Bo Magma
----------------

In [10]:
enum RPS {
    ROCK = '✊',
    PAPER = '✋',
    SCISSORS = '✌'
}

undefined

In [11]:
function shoot(a: RPS, b: RPS): RPS {
    return a === RPS.ROCK && b === RPS.SCISSORS ? RPS.ROCK :
           a === RPS.PAPER && b === RPS.ROCK ? RPS.PAPER :
           a === RPS.SCISSORS && b === RPS.PAPER ? RPS.SCISSORS :
           b;
}

undefined

In [12]:
const ALL_RPS = [RPS.ROCK, RPS.PAPER, RPS.SCISSORS];

undefined

In [13]:
ALL_RPS.forEach(a => console.log(`${a} * ${RPS.ROCK} = ${shoot(a, RPS.ROCK)}`));

✊ * ✊ = ✊
✋ * ✊ = ✋
✌ * ✊ = ✊


undefined

In [14]:
ALL_RPS.forEach(a => console.log(`${a} * ${RPS.PAPER} = ${shoot(a, RPS.PAPER)}`));

✊ * ✋ = ✋
✋ * ✋ = ✋
✌ * ✋ = ✌


undefined

In [15]:
ALL_RPS.forEach(a => console.log(`${a} * ${RPS.SCISSORS} = ${shoot(a, RPS.SCISSORS)}`));

✊ * ✌ = ✊
✋ * ✌ = ✌
✌ * ✌ = ✌


undefined

## Other Magmas ##

- Addition over natural numbers

- Subtraction over integers

Semigroup
---------

- append

### Semigroup Axioms ###

- Totality
- Associativity

#### Associativity ####

$$(x \odot y) \odot z = x \odot (y \odot z) \forall x, y, z \in S.$$

In [16]:
interface Semigroup {
    concat(a: Semigroup): Semigroup;
}

undefined

In [17]:
function sappend(a: Semigroup, b: Semigroup): Semigroup {
    return a.concat(b);
}

undefined

In [18]:
sappend('hello', sappend('world', '!'));

'helloworld!'

In [19]:
sappend(sappend('hello', 'world'), '!');

'helloworld!'

In [ ]:
sappend(['hello'], ['world']);

Error: Line 1, Character 9
sappend(['hello'], ['world']);
________^
TS2345: Argument of type 'string[]' is not assignable to parameter of type 'Semigroup'.
  Types of property 'concat' are incompatible.
    Type '{ (...items: ConcatArray<string>[]): string[]; (...items: (string | ConcatArray<string>)[]): string[]; }' is not assignable to type '(a: Semigroup) => Semigroup'.
      Types of parameters 'items' and 'a' are incompatible.
        Type 'Semigroup' is missing the following properties from type 'ConcatArray<string>': length, join, slice

In [21]:
interface Array<T> {
    concat(x: Array<T>): Array<T>;
}

undefined

In [22]:
sappend(['hello'], ['world']);

[ 'hello', 'world' ]

What about numbers?

In [23]:
class SSum implements Semigroup {
    constructor(readonly value: number) {}
    concat(other: SSum) {
        return new SSum(other.value + this.value);
    }
}

undefined

In [24]:
class SProduct implements Semigroup {
    constructor(readonly value: number) {}
    concat(other: SProduct) {
        return new SProduct(other.value * this.value);
    }
}

undefined

In [35]:
sappend(new SProduct(2), new SProduct(5))

SProduct { value: 10 }

In [26]:
sappend(new SSum(1), new SSum(2));

SSum { value: 3 }

How else can we append numbers?

- NumberProduct ?

- NumberSubtraction ?

Is subtraction associative?

In [27]:
(1 - 2) - 3

-4

In [28]:
1 - (2 -3)

2

Application of Semigroups
-------------------------

In [29]:
function stimes(s: Semigroup, n: number, start=s): Semigroup {
    return n <= 1 ? s : stimes(s.concat(start), n - 1, start)
}

undefined

In [30]:
stimes('x', 80);

'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [31]:
stimes(new SSum(3), 5);

SSum { value: 15 }

In [36]:
stimes(new SProduct(2), 8)

SProduct { value: 256 }

What about zero times?

Monoid
------

- Has sappend and all of its rules.
- Has sempty

In [32]:
interface Monoid extends Semigroup {
    empty(): Semigroup;
}

undefined

In [33]:
class MSum extends SSum implements Monoid {
    empty() {
        return new MSum(0);
    }
}

undefined

In [ ]:
function mtimes(s: Monoid, n: number, start=s): Monoid {
    return n === 1 ? s : n === 0 ? s.empty() : (mtimes(s.concat(start) as Monoid, n - 1, start) as Monoid);
}

Error: Line 2, Character 5
    return n === 1 ? s : n === 0 ? s.empty() : (mtimes(s.concat(start) as Monoid, n - 1, start) as Monoid);
____^
TS2741: Property 'empty' is missing in type 'Semigroup' but required in type 'Monoid'.